In [13]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_pinecone import PineconeVectorStore
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import yaml
import os
import zipfile

In [14]:
with open('config.yaml', 'r') as config_file:
    config = yaml.safe_load(config_file)
os.environ['PINECONE_API_KEY'] = config['PINECONE_API_KEY']
os.environ['OPENAI_API_KEY'] = config['OPENAI_API_KEY']

In [15]:
zip_file_path = 'documentos.zip'
extracted_folder_path = 'docs'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

documents = []
for filename in os.listdir(extracted_folder_path):
    if filename.endswith(".pdf"):
        file_path = os.path.join(extracted_folder_path, filename)
        loader = PyMuPDFLoader(file_path)
        documents.extend(loader.load())

In [16]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  
    chunk_overlap=100,
    length_function=len
)
chunks = text_splitter.create_documents([doc.page_content for doc in documents])


In [17]:
print(chunks)

[Document(metadata={}, page_content='Número do Processo: 175543\nViolação de normas ambientais pela Empresa de Construção durante um grande projeto de\ndesenvolvimento. Impactos causados incluíram a contaminação de um rio local e a destruição de\náreas de floresta protegida. A defesa da empresa argumentou que todas as medidas necessárias\nforam tomadas para minimizar os impactos. O julgamento resultou em uma multa de $500,000 e a\nordem para a restauração das áreas afetadas.'), Document(metadata={}, page_content='Número do Processo: 674306\nReclamações de condições de trabalho inadequadas e demissão sem justa causa. O funcionário\nalegou que a Empresa Z não forneceu um ambiente de trabalho seguro e o demitiu em retaliação\npor relatar essas condições. A Empresa Z defendeu que o funcionário foi demitido por desempenho\ninsatisfatório. O tribunal decidiu a favor do funcionário, ordenando que a Empresa Z pague uma\nindenização de $50,000 e melhore as condições de trabalho.'), Document(met

Texto original: "Este é um documento muito longo com muitas informações importantes sobre processos judiciais..."

Chunk 1: "Este é um documento muito longo com muitas informações importantes..." (1000 chars)

Chunk 2: "...informações importantes sobre processos judiciais e suas decisões..." (últimos 100 chars do chunk 1 + novos 900 chars)

In [18]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [19]:
print(embeddings)

client=<openai.resources.embeddings.Embeddings object at 0x000001FD6AC1B6E0> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x000001FD6C994EC0> model='text-embedding-ada-002' dimensions=None deployment='text-embedding-ada-002' openai_api_version=None openai_api_base=None openai_api_type=None openai_proxy=None embedding_ctx_length=8191 openai_api_key=SecretStr('**********') openai_organization=None allowed_special=None disallowed_special=None chunk_size=1000 max_retries=2 request_timeout=None headers=None tiktoken_enabled=True tiktoken_model_name=None show_progress_bar=False model_kwargs={} skip_empty=False default_headers=None default_query=None retry_min_seconds=4 retry_max_seconds=20 http_client=None http_async_client=None check_embedding_ctx_length=True


In [20]:
index_name = "llm"
vector_store = PineconeVectorStore.from_documents(
    documents=chunks, 
    embedding=embeddings, 
    index_name=index_name
)

In [21]:
query_1 = '''Responda apenas com base no input fornecido. Qual o número do processo que trata de Violação
de normas ambientais pela Empresa de Construção?'''

query_2 = 'Responda apenas com base no input fornecido. Qual foi a decisão no caso de fraude financeira?'

query_3 = 'Responda apenas com base no input fornecido. Quais foram as alegações no caso de negligência médica?'

query_4 = 'Responda apenas com base no input fornecido. Quais foram as alegações no caso de Número do Processo: 822162' #disputa contratual

In [22]:
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 3})

In [30]:
print(retriever.get_graph())

Graph(nodes={'94c8eb20fafc4ae1979a1442f4670393': Node(id='94c8eb20fafc4ae1979a1442f4670393', name='VectorStoreRetrieverInput', data=<class 'langchain_core.vectorstores.base.VectorStoreRetrieverInput'>, metadata=None), '2b51bba5dc3c4824b9ba69a5f8a9d794': Node(id='2b51bba5dc3c4824b9ba69a5f8a9d794', name='VectorStoreRetriever', data=VectorStoreRetriever(tags=['PineconeVectorStore', 'OpenAIEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x000001FD6953A270>, search_kwargs={'k': 3}), metadata=None), 'ec0a9f0558aa42ee85b249a3ee8f4aba': Node(id='ec0a9f0558aa42ee85b249a3ee8f4aba', name='VectorStoreRetrieverOutput', data=<class 'langchain_core.vectorstores.base.VectorStoreRetrieverOutput'>, metadata=None)}, edges=[Edge(source='94c8eb20fafc4ae1979a1442f4670393', target='2b51bba5dc3c4824b9ba69a5f8a9d794', data=None, conditional=False), Edge(source='2b51bba5dc3c4824b9ba69a5f8a9d794', target='ec0a9f0558aa42ee85b249a3ee8f4aba', data=None, conditional=False)])


In [23]:
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.2)

In [24]:
# Criando um template de prompt para o RAG
template = """Use o seguinte contexto para responder a pergunta. Se você não souber a resposta, apenas diga que não sabe, não tente inventar uma resposta.

Contexto: {context}

Pergunta: {question}

Resposta:"""

prompt = ChatPromptTemplate.from_template(template)

# Criando a chain usando LCEL
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [25]:
answer_1 = rag_chain.invoke(query_1)
answer_2 = rag_chain.invoke(query_2)
answer_3 = rag_chain.invoke(query_3)
answer_4 = rag_chain.invoke(query_4)

In [26]:
print('Pergunta: ', query_1)
print('Resultado: ', answer_1, '\n')

print('Pergunta: ', query_2)
print('Resultado: ', answer_2, '\n')

print('Pergunta: ', query_3)
print('Resultado: ', answer_3, '\n')

print('Pergunta: ', query_4)
print('Resultado: ', answer_4)

Pergunta:  Responda apenas com base no input fornecido. Qual o número do processo que trata de Violação
de normas ambientais pela Empresa de Construção?
Resultado:  O número do processo é 175543. 

Pergunta:  Responda apenas com base no input fornecido. Qual foi a decisão no caso de fraude financeira?
Resultado:  A decisão no caso de fraude financeira foi de dez anos de prisão para João Almeida e a restituição de $1,000,000 ao Banco Nacional. 

Pergunta:  Responda apenas com base no input fornecido. Quais foram as alegações no caso de negligência médica?
Resultado:  As alegações no caso de negligência médica foram de que Dr. Pedro Sousa não realizou os procedimentos médicos com o cuidado necessário, resultando em complicações graves, incluindo uma cirurgia mal-sucedida que levou a infecções adicionais. 

Pergunta:  Responda apenas com base no input fornecido. Quais foram as alegações no caso de Número do Processo: 822162
Resultado:  As alegações no caso do Número do Processo 822162 for